## Getting Started with Terality

In this notebook, you'll get to experiment with Terality library from installing the library to performing Data Wrangling and Feature Engineering.

### Installing and confguring the environment

First create a free account on Terality. Head over to the [Terality website](https://www.terality.com/) and simply create your account.

Once you successfully create the account, you will be directed to the Dashboard.

<img src="images/dashboard.jpg" align="center"/>

Follow the steps below to set up your environment. (Alternatively, you may follow the Quickstart instructions in the Terality Dashboard).

1. The first step is to install the Terality library. To do so, execute the command below on your terminal.

```
pip install --upgrade terality
```

2. Next we have to generate and configure an API key. Navigate to the Terality Dashboard and click on Click here to generate an API key. An API key will be generated and displayed.

<img src="images/get_api_key.jpg" align="center"/>

(Please note that you won't be able to see the API key once you navigate from the Dashboard. Therefore, if you are not configuring immediately, save the API key in some secured location).
    
3. Execute the command below on your terminal to configure the Terality library. Replace the emailaddress@example.com with the email address you used for signup.

```
terality account configure --email emailaddress@example.com
```

Once you execute the above command, you will be prompt to enter **Your Terality API key**. Enter the API key generated in Step 2.Upon entering the key, a confirmation message will be displayed. *Terality account succesfully configured on this system.*



## Real-word Example: Data Wrangling and Feature Engineering

To get hold of the benefits extended by the Terality library, let's get our hands dirty with a real-world example.
For the demonstration, I have chosen to use the NYC Taxi dataset which is available in [AWS S3 open data registry](https://registry.opendata.aws/nyc-tlc-trip-records-pds/).

The S3 bucket has data for Yellow taxi trips as well as for Green taxi trips. In our demonstration we'll be using the Yellow taxi trip data for the year of 2020.

We'll be preparing our data for modeling but performing preprocessing steps.

Let's first import the Terality library and get our hands dirty with the preparation

### Import neccessary libraries

In [1]:
import terality as te

You are using version 0.12.25 of the Terality client, but version 0.12.26 is available. Consider upgrading your version to get the latest fixes and features.


### Import Data

In [2]:
%%time
data = None

for month in range(1,13):
    s3_data_file = f"s3://nyc-tlc/trip data/yellow_tripdata_2020-{str(month).zfill(2)}.csv"
    
    if data is not None:
        temp_df = te.read_csv(s3_data_file)
        data = data.append(temp_df)
    else:
        data = te.read_csv(s3_data_file)

Wall time: 3min 50s


### Explore Data

In this section, we will explore and modify dataset as per the our requirment for the further analysis of the problem.

As a first step, let's take a copy of the data and then start the preparation. The reason for doing this is, in case you make any undesired change to the dataframe, then you might need to load again from the S3 file.

Instead, if you have a copy of the original dataframe, you can take another copy and keep on processing. Execute the cell below to take a copy of the data. You may execute this anytime you require the original dataframe in the later time.

In [3]:
# Take a copy of the orginal data
taxi_data = data.copy()

While importing the data, the data was loaded from multiple files and merged together. Therefore, the rows will have duplicate indices. To make it unique, let's reset the index.

In [4]:
# Reset index
taxi_data = taxi_data.reset_index()

Let's take a look at what the data looks like.

In [5]:
taxi_data.info()

<class 'terality.DataFrame'>
Index: 24648499 entries, 0 to 24648498
Data columns (total 19 columns):
 #   Column                 Dtype
---  ------                 -----
 0   index                  int64
 1   VendorID               float64
 2   tpep_pickup_datetime   object
 3   tpep_dropoff_datetime  object
 4   passenger_count        float64
 5   trip_distance          float64
 6   RatecodeID             float64
 7   store_and_fwd_flag     object
 8   PULocationID           int64
 9   DOLocationID           int64
10   payment_type           float64
11   fare_amount            float64
12   extra                  float64
13   mta_tax                float64
14   tip_amount             float64
15   tolls_amount           float64
16   improvement_surcharge  float64
17   total_amount           float64
18   congestion_surcharge   float64
dtypes: int64(3), float64(13), object(3)
memory usage: 7.9 GB (run with deep=True)


Observations

- There are **24648499** records in our dataset.
- Data consist of **19** columns.
- The data type of each column is reflected next to it.

In [6]:
taxi_data.head()

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


Let's try to understand what each column is

**Feature details:**

- `VendorID`: This is code indicating the record provider.
- `tpep_pickup_datetime`: The date and time when the meter was engaged (started). 
- `tpep_dropoff_datetime`: The date and time when the meter was disengaged (ended).
- `passenger_count`: The number of passengers in the vehicle. This is a driver entered value.
- `trip_distance`: The elapsed trip distance in miles reported by the taximeter.
- `RateCodeID`: The final rate code in effect at the end of the trip. 1= Standard rate, 2=JFK, 3=Newark, 4=Nassau or Westchester, 5=Negotiated fare, 6=Group ride
- `store_and_fwd_flag`: This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server. Y= store and forward trip N= not a store and forward trip
- `PULocationID`: TLC Taxi Zone in which the taximeter was engaged.
- `DOLocationID`: TLC Taxi Zone in which the taximeter was disengaged.
- `payment_type`: A numeric code signifying how the passenger paid for the trip. 1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided trip
- `fare_amount`: The time and distance fare calculated by the meter.
- `extra`: Miscellaneous extras and surcharges. Currently, this only includes the \$0.50 and \\$1 rush hour and overnight charges respectively.
- `mta_tax`: \$0.50 MTA tax that is automatically triggered based on the metered rate in use.
- `tip_amount`: Tip amount. This field is automatically populated for credit card tips. Cash tips are not included.
- `tolls_amount`: Total amount of all tolls paid in trip. 
- `improvement_surcharge`: \$0.30 improvement surcharge assessed on hailed trips at the flag drop. The improvement surcharge began being levied in 2015.
- `congestion_surcharge`: Surcharge due to congestion


**Label**
- `total_amount`: The total amount charged to passengers. Does not include cash tips.

Now that we understand what each column represent, let's again have a look on the datatypes and see if we need to make any modifications to it. To have glance on the datatype of each column execute the cell below.

In [7]:
taxi_data.dtypes

index                      int64
VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

Here we can see the `tpep_pickup_datetime` and `tpep_dropoff_datetime` are of the type *Object*. Ideally, both of these features should be of datatype *datetime*.

Therefore, we will now convert the data type of those 2 columns to *datetime*.

In [8]:
taxi_data['tpep_pickup_datetime'] = te.to_datetime(taxi_data['tpep_pickup_datetime'])
taxi_data['tpep_dropoff_datetime'] = te.to_datetime(taxi_data['tpep_dropoff_datetime'])

It's always good to check on the change we make. Let's now verify the data type change

In [9]:
taxi_data.dtypes

index                             int64
VendorID                        float64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                    float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

We can see that the data type of `lpep_pickup_datetime` and `lpep_dropoff_datetime` are changed to *datetime*.

Next, let's move to the next step of handing missing values.

#### Handling null values

To have a glance at the null values in the dataset, execute the cell below.

In [10]:
taxi_data.isnull().sum()

index                         0
VendorID                 809568
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          809568
trip_distance                 0
RatecodeID               809568
store_and_fwd_flag       809568
PULocationID                  0
DOLocationID                  0
payment_type             809568
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge          0
dtype: int64

Usually in any problem, missing values can be handled by any of the following ways

1. Deleting Rows: Deleting all the records which have null values
2. Replacing with Mean/Mode or Median
3. Assigning new category
4. Predicting the missing value
5. Using algorithms which support missing values

Before deciding on which we are about to do with the missing values, let's observe the count of missing values.

It can be observed that *VendorID*, *passenger_count*, *RatecodeID*, *store_and_fwd_flag*, and *payment_type* columns have null values.

- All columns: About 3% null values

Given that we only have 3% of data having null values in the 5 columns, we can drop the respective rows.

Let's drop the rows with null values and view the null sum again.

In [11]:
# Drop the rows with null values
taxi_data.dropna(inplace=True)
taxi_data.isnull().sum()

index                    0
VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
dtype: int64

Now our dataset does not contain any null values.

### Feature Engineering

Feature engineering refers to the process of creating new input variables from the available data.

Let's try very simple feature engineering with the data.

Based on our analysis above, using the `tpep_pickup_datetime` and `tpep_dropoff_datetime` features, we can create more features.

i.e., the pickup and drop day, pickup and drop hour, and pickup and drop month.

In [13]:
taxi_data['pickup_day']=taxi_data['tpep_pickup_datetime'].dt.day_name()
taxi_data['dropoff_day']=taxi_data['tpep_dropoff_datetime'].dt.day_name()

In [14]:
taxi_data['pickup_day_no']=taxi_data['tpep_pickup_datetime'].dt.weekday
taxi_data['dropoff_day_no']=taxi_data['tpep_dropoff_datetime'].dt.weekday

In [15]:
taxi_data['pickup_hour']=taxi_data['tpep_pickup_datetime'].dt.hour
taxi_data['dropoff_hour']=taxi_data['tpep_dropoff_datetime'].dt.hour

In [16]:
taxi_data['pickup_month']=taxi_data['tpep_pickup_datetime'].dt.month
taxi_data['dropoff_month']=taxi_data['tpep_dropoff_datetime'].dt.month

## References

1. https://github.com/aws-samples/cloud-experiments/tree/master/experiments/notebooks/exploring-data
2. https://www.kaggle.com/nitin194/nyc-taxi-trip-duration-prediction
3. https://registry.opendata.aws/nyc-tlc-trip-records-pds/